In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import os
import re
import datetime
import pandas as pd
import json
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [2]:
# Spark Core
sc = SparkContext('local')
spark = SparkSession(sc)

In [3]:
spark = SparkSession.builder.appName('Read All CSV Files in Directory').getOrCreate()

# 讀取所有csv結尾的檔案
file2 = spark.read.csv('./*.csv', sep=',', inferSchema=True, header=True)
 
# 使用Pandas
df1 = file2.toPandas()

# 檢查資料頭尾
display(df1.head())
display(df1.tail())

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號
0,0,The villages and towns urban district,transaction sign,land sector position building sector house num...,land shifting total area square meter,the use zoning or compiles and checks,the non-metropolis land use district,non-metropolis land use,transaction year month and day,transaction pen number,...,the berth category,berth shifting total area square meter,the berth total price NTD,the note,serial number,NaN,NaN,NaN,None,None
1,1,板橋區,房地(土地+建物)+車位,新北市板橋區光復街54號7樓,23.04,住,None,None,1071117,土地1建物1車位1,...,坡道平面,36.35,0,含增建或未登記建物。,RPSNMLNKKHOFFAF97CA,91.21,0.48,11.18,None,None
2,2,板橋區,房地(土地+建物),新北市板橋區民生路三段246號16樓,7.83,住,None,None,1071214,土地2建物1車位0,...,None,0.0,0,含增建或未登記建物。,RPTNMLNKKHOFFAF08CA,109.02,0.00,13.33,None,None
3,3,土城區,土地,瑞興段473-1地號,713.59,其他,None,None,1080114,土地1建物0車位0,...,None,0.0,0,包含公共設施保留地之交易。；,RPOPMLNKKHOFFAF77CA,0.00,0.00,0.00,None,None
4,4,土城區,房地(土地+建物),新北市土城區學府路一段１２２之３號十七樓,5.75,住,None,None,1071224,土地1建物1車位0,...,None,0.0,0,None,RPUOMLLKKHOFFAF08CA,19.72,0.00,3.17,None,None


,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號
49642,5591,文山區,房地(土地+建物)+車位,臺北市文山區汀州路四段273號九樓,35.8,住,None,None,1080422,土地1建物1車位1,...,坡道平面,0.0,0,向政府機關承購之案件。,RPOOMLOLNHOFFAA67CA,96.77,0.00,11.11,None,None
49643,5592,文山區,房地(土地+建物),臺北市文山區辛亥路七段２０巷１弄２號,24.61,住,None,None,1080403,土地1建物1車位0,...,None,0.0,0,親友、員工或其他特殊關係間之交易。；二親等間買賣,RPPNMLTLNHOFFAA18CA,51.29,0.27,0.00,None,None
49644,5593,文山區,房地(土地+建物)+車位,臺北市文山區忠順街一段４８巷５弄１０號,54.64,住,None,None,1080326,土地1建物2車位2,...,坡道平面,85.3,0,兩戶打通,RPUNMLTKOHOFFAA68CA,143.11,0.00,15.19,None,None
49645,5594,中正區,房地(土地+建物)+車位,臺北市中正區?m門街７７號１２樓,12.52,商,None,None,1080412,土地2建物1車位2,...,坡道平面,69.42,5500000,工程款減：1830000,RPSNMLQJOHOFFAA08CA,58.31,0.86,8.31,None,None
49646,5595,中正區,房地(土地+建物),臺北市中正區?m門街７７號１２樓之１,10.25,商,None,None,1080412,土地2建物1車位0,...,None,0.0,0,工程款減：1480000,RPTNMLQJOHOFFAA18CA,50.99,1.38,3.55,None,None


In [4]:
# 驗證台中市的資料
df1[df1['鄉鎮市區'] == '豐原區'].tail()

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號
34809,10674,豐原區,房地(土地+建物),臺中市豐原區中正路６９５巷３１號四樓,9.5,住,None,None,1080409,土地1建物1車位0,...,None,0.0,0,None,RPVNMLOKNHOFFDB58CA,66.53,6.77,0.0,None,None
34811,10676,豐原區,房地(土地+建物),臺中市豐原區鐮村路６７２巷１２號,69.66,None,鄉村區,乙種建築用地,1080401,土地1建物1車位0,...,None,0.0,0,含增建或未登記建物。；,RPTOMLKKNHOFFDB08CA,85.80,0.00,0.0,None,None
34814,10679,豐原區,房地(土地+建物)+車位,臺中市豐原區育仁路１１４巷１４號五樓,24.95,住,None,None,1080408,土地1建物1車位1,...,升降機械,0.0,0,None,RPTNMLNJOHOFFDB28CA,87.57,7.88,0.0,None,None
34816,10681,豐原區,房地(土地+建物),臺中市豐原區新生北路２０６號十樓之５,3.72,商,None,None,1080331,土地1建物1車位0,...,None,0.0,0,None,RPROMLQJOHOFFDB48CA,20.73,2.93,0.0,None,None
35010,10875,豐原區,土地,南村段620地號,105.19,住,None,None,1080416,土地1建物0車位0,...,None,0.0,0,親友、員工或其他特殊關係間之交易。；二親等買賣未訂私契,RPRNMLNKNHOFFDB18DA,0.00,0.00,0.0,None,None


In [5]:
# 驗證桃園市的資料
df1[df1['鄉鎮市區'] == '中壢區'].tail()

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號
43951,8922,中壢區,房地(土地+建物),桃園市中壢區功學路５２號七樓,16.8,住,None,None,1080320,土地1建物1車位0,...,None,0.0,0,None,RPUPMLNKNHOFFBH98CA,71.23,2.88,8.27,None,None
43952,8923,中壢區,房地(土地+建物)+車位,桃園市中壢區高鐵站前西路二段１７號四樓,31.8,商,None,None,1080417,土地1建物1車位1,...,坡道平面,29.09,1400000,None,RPSQMLNKNHOFFBH88CA,132.62,3.00,14.95,None,None
43953,8924,中壢區,房地(土地+建物)+車位,桃園市中壢區延平路９０巷１１０號八樓,31.33,住,None,None,1080403,土地1建物1車位1,...,坡道平面,29.84,0,None,RPVOMLNKNHOFFBH98CA,60.08,2.46,7.74,None,None
43954,8925,中壢區,房地(土地+建物)+車位,桃園市中壢區青雲路１０８號六樓之６,18.15,商,None,None,1080309,土地1建物1車位1,...,坡道平面,36.15,1350000,None,RPROMLNKNHOFFBH58CA,66.02,8.66,5.57,None,None
43973,8944,中壢區,房地(土地+建物)+車位,桃園市中壢區中山東路三段６１號１３樓之５,10.74,商,None,None,1080331,土地1建物1車位1,...,坡道平面,40.11,1000000,含增建或未登記建物。,RPOPMLNKNHOFFBH48DA,83.10,5.03,10.62,None,None


In [6]:
# 驗證新北市的資料
df1[df1['鄉鎮市區'] == '中和區'].tail()

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號
12829,12829,中和區,房地(土地+建物),新北市中和區圓通路２６５巷２３號五樓,29.62,住,None,None,1080415,土地2建物1車位0,...,None,0.0,0,含增建或未登記建物。；,RPVOMLKKNHOFFHF09CA,84.34,0.00,10.83,None,None
12830,12830,中和區,房地(土地+建物)+車位,新北市中和區中山路二段３３２巷１７號八樓,18.4,工,None,None,1080417,土地1建物1車位1,...,坡道平面,33.57,1800000,None,RPPPMLKKNHOFFHF58CA,59.92,0.00,18.82,None,None
12831,12831,中和區,房地(土地+建物),新北市中和區中山路二段３３２巷４７號九樓,16.53,工,None,None,1080318,土地1建物1車位0,...,None,0.0,0,None,RPUPMLKKNHOFFHF09CA,74.35,0.00,4.86,None,None
12832,12832,中和區,房地(土地+建物)+車位,新北市中和區捷運路２５號十九樓,17.15,住,None,None,1080408,土地1建物1車位1,...,坡道平面,41.2,1900000,None,RPWNMLNKNHOFFHF76CA,78.81,0.39,9.85,None,None
12834,12834,中和區,房地(土地+建物)+車位,新北市中和區捷運路２５號十八樓,17.15,住,None,None,1080408,土地1建物1車位1,...,坡道平面,41.2,1900000,None,RPTNMLNKNHOFFHF09CA,78.81,0.39,9.85,None,None


In [7]:
# 驗證高雄市的資料
df1[df1['鄉鎮市區'] == '小港區'].tail()

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號
24068,11108,小港區,房地(土地+建物)+車位,高雄市小港區華昌路１９號十樓,21.43,住,None,None,1080430,土地2建物1車位1,...,坡道平面,22.82,0,含裝潢,RPSOMLQJOHOFFCE78CA,64.70,11.59,0.0,None,None
24069,11109,小港區,房地(土地+建物),高雄市小港區中安路９３５號九樓,13.2,住,None,None,1080402,土地1建物1車位0,...,None,0.0,0,None,RPUOMLNLNHOFFCE78CA,99.97,16.13,0.0,None,None
24070,11110,小港區,房地(土地+建物),高雄市小港區宏文路２０２號,29.98,住,None,None,1080401,土地1建物1車位0,...,None,0.0,0,None,RPRNMLNLNHOFFCE38CA,150.37,26.34,0.0,None,None
24071,11111,小港區,房地(土地+建物),高雄市小港區大業北路４３４之１號,26.25,商,None,None,1080321,土地1建物1車位0,...,None,0.0,0,None,RPVRMLNJOHOFFCE09CA,168.84,8.45,0.0,None,None
24130,11170,小港區,房地(土地+建物),高雄市小港區民益路77巷7弄3之4號,31.3,住,None,None,1080502,土地1建物1車位0,...,None,0.0,0,親友、員工或其他特殊關係間之交易。,RPPNMLRJNHOFFCE68CD,91.02,9.36,0.0,None,None


In [8]:
# 驗證台北市的資料
df1[df1['鄉鎮市區'] == '大安區'].tail()

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號
49577,5526,大安區,房地(土地+建物),臺北市大安區復興南路二段125巷32號,33.75,住,None,None,1080228,土地1建物1車位0,...,None,0.0,0,含增建或未登記建物。；,RPROMLRKNHOFFFA68CA,95.40,0.00,0.00,None,None
49578,5527,大安區,房地(土地+建物),臺北市大安區敦化南路二段２６７號九樓之１,3.6,其他,None,None,1080314,土地1建物1車位0,...,None,0.0,0,親友、員工或其他特殊關係間之交易。；,RPWRMLOKNHOFFFA56CA,42.83,0.00,0.00,None,None
49579,5528,大安區,土地,通化段六小段653-1地號,0.66,其他,None,None,1080502,土地1建物0車位0,...,None,0.0,0,親友、員工或其他特殊關係間之交易。；,RPOSMLOKNHOFFFA48CA,0.00,0.00,0.00,None,None
49580,5529,大安區,房地(土地+建物),臺北市大安區延吉街１７１號三樓,15.14,住,None,None,1080420,土地1建物1車位0,...,None,0.0,0,有民情風俗因素之交易。；商場,RPQNMLOKNHOFFFA18CA,52.19,9.53,6.14,None,None
49581,5530,大安區,房地(土地+建物),臺北市大安區復興南路一段313巷11號四樓,37.25,住,None,None,1080329,土地1建物1車位0,...,None,0.0,0,None,RPSPMLQKNHOFFFA78CA,91.91,0.00,16.83,None,None


In [9]:
# Columns
df1.columns

Index(['_c0', '鄉鎮市區', '交易標的', '土地位置建物門牌', '土地移轉總面積平方公尺', '都市土地使用分區',
       '非都市土地使用分區', '非都市土地使用編定', '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態',
       '主要用途', '主要建材', '建築完成年月', '建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳',
       '建物現況格局-衛', '建物現況格局-隔間', '有無管理組織', '總價元', '單價元平方公尺', '車位類別',
       '車位移轉總面積(平方公尺)', '車位總價元', '備註', '編號', '主建物面積', '附屬建物面積', '陽台面積', '電梯',
       '移轉編號'],
      dtype='object')

In [10]:
# 觀察值
df1['主要用途'].unique()

array(['main use', '住家用', None, '見其他登記事項', '商業用', '工商用', '住商用', '住工用',
       '停車空間', '工業用', '見使用執照', '農舍', '辦公室', '見其它登記事項', '國民住宅', '集合住宅',
       '農業用'], dtype=object)

In [11]:
# 觀察值
df1['建物型態'].unique()

array(['building state', '華廈(10層含以下有電梯)', '住宅大樓(11層含以上有電梯)', '其他',
       '套房(1房1廳1衛)', '公寓(5樓含以下無電梯)', '店面(店鋪)', '透天厝', '辦公商業大樓', '廠辦',
       '工廠', '農舍', '倉庫'], dtype=object)

In [12]:
# 觀察值
df1['總樓層數'].unique()

array(['total floor number', '九層', '二十六層', None, '十八層', '十二層', '四層',
       '四十層', '十九層', '二十二層', '六層', '十五層', '二十八層', '十層', '五層', '三十三層',
       '八層', '二十層', '二十七層', '十一層', '三層', '十四層', '七層', '十六層', '二層', '十七層',
       '二十九層', '二十三層', '十三層', '二十五層', '四十一層', '三十四層', '二十一層', '二十四層',
       '三十七層', '一層', '三十二層', '000', '三十一層', '四十三層', '三十六層', '四十二層', '三十層',
       '三十五層', '三十八層', '四十六層', '三十九層', '八十五層', '五十層'], dtype=object)

In [13]:
# 因為樓層總數為中文
# 欲將其轉成數字 
# 之後方便篩選
floor = df1['總樓層數'].unique().tolist()

In [14]:
# 用dict紀錄英文轉數字
floor_dict = {}

In [15]:
floor_dict[floor[0]] = -1
floor_dict[floor[1]] = 9
floor_dict[floor[2]] = 26
floor_dict[floor[3]] = 0
floor_dict[floor[4]] = 18
floor_dict[floor[5]] = 12
floor_dict[floor[6]] = 4
floor_dict[floor[7]] = 40
floor_dict[floor[8]] = 19
floor_dict[floor[9]] = 22
floor_dict[floor[10]] = 6
floor_dict[floor[11]] = 15
floor_dict[floor[12]] = 28
floor_dict[floor[13]] = 10
floor_dict[floor[14]] = 5
floor_dict[floor[15]] = 33
floor_dict[floor[16]] = 8
floor_dict[floor[17]] = 20
floor_dict[floor[18]] = 27
floor_dict[floor[19]] = 11
floor_dict[floor[20]] = 3
floor_dict[floor[21]] = 14
floor_dict[floor[22]] = 7
floor_dict[floor[23]] = 16
floor_dict[floor[24]] = 2
floor_dict[floor[25]] = 17
floor_dict[floor[26]] = 29
floor_dict[floor[27]] = 23
floor_dict[floor[28]] = 13
floor_dict[floor[29]] = 25
floor_dict[floor[30]] = 41
floor_dict[floor[31]] = 34
floor_dict[floor[32]] = 21
floor_dict[floor[33]] = 24
floor_dict[floor[34]] = 37
floor_dict[floor[35]] = 1
floor_dict[floor[36]] = 32
floor_dict[floor[37]] = 0
floor_dict[floor[38]] = 31
floor_dict[floor[39]] = 43
floor_dict[floor[40]] = 36
floor_dict[floor[41]] = 42
floor_dict[floor[42]] = 30
floor_dict[floor[43]] = 35
floor_dict[floor[44]] = 38
floor_dict[floor[45]] = 46
floor_dict[floor[46]] = 39
floor_dict[floor[47]] = 85
floor_dict[floor[48]] = 50

In [16]:
# Print
floor_dict

{'total floor number': -1,
 '九層': 9,
 '二十六層': 26,
 None: 0,
 '十八層': 18,
 '十二層': 12,
 '四層': 4,
 '四十層': 40,
 '十九層': 19,
 '二十二層': 22,
 '六層': 6,
 '十五層': 15,
 '二十八層': 28,
 '十層': 10,
 '五層': 5,
 '三十三層': 33,
 '八層': 8,
 '二十層': 20,
 '二十七層': 27,
 '十一層': 11,
 '三層': 3,
 '十四層': 14,
 '七層': 7,
 '十六層': 16,
 '二層': 2,
 '十七層': 17,
 '二十九層': 29,
 '二十三層': 23,
 '十三層': 13,
 '二十五層': 25,
 '四十一層': 41,
 '三十四層': 34,
 '二十一層': 21,
 '二十四層': 24,
 '三十七層': 37,
 '一層': 1,
 '三十二層': 32,
 '000': 0,
 '三十一層': 31,
 '四十三層': 43,
 '三十六層': 36,
 '四十二層': 42,
 '三十層': 30,
 '三十五層': 35,
 '三十八層': 38,
 '四十六層': 46,
 '三十九層': 39,
 '八十五層': 85,
 '五十層': 50}

In [17]:
# 新增欄位
# 樓層數的阿拉伯數字
df1['總樓層數(數字)'] = df1['總樓層數'].map(floor_dict)

In [18]:
# 轉換型態
df1 = df1.astype({'總樓層數(數字)': int})

In [19]:
# 篩選條件
df1_result = df1[(df1['主要用途'] == '住家用') & (df1['建物型態'].str.match('^住宅大樓*')== True) & (df1['總樓層數(數字)']>=13)]

In [20]:
# Print 看是否都高於13樓
df1_result['總樓層數(數字)'].unique()

array([26, 40, 15, 20, 27, 14, 16, 17, 23, 13, 28, 25, 41, 18, 21, 24, 29,
       37, 22, 19, 32, 31, 43, 33, 36, 30, 38, 34, 46, 35, 39, 42])

In [21]:
# Print dataframe
df1_result

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,總樓層數(數字)
2,2,板橋區,房地(土地+建物),新北市板橋區民生路三段246號16樓,7.83,住,None,None,1071214,土地2建物1車位0,...,0.0,0,含增建或未登記建物。,RPTNMLNKKHOFFAF08CA,109.02,0.00,13.33,None,None,26
8,8,板橋區,房地(土地+建物)+車位,新北市板橋區文化路一段３６０號二十三樓之１３,6.88,住,None,None,1071215,土地1建物2車位3,...,101.1,0,None,RPUOMLMLKHOFFAF28CA,45.86,0.00,10.06,None,None,40
151,151,新店區,房地(土地+建物)+車位,新北市新店區中央路127巷13號2樓之2,13.97,住,None,None,1071207,土地1建物1車位1,...,39.01,0,含增建或未登記建物。；,RPVNMLMKKHOFFCF38CA,87.37,1.92,11.74,None,None,15
160,160,淡水區,房地(土地+建物)+車位,新北市淡水區沙崙路一段１２２號七樓,21.9,住,None,None,1071214,土地1建物1車位2,...,56.22,0,建商與地主合建案。；,RPRNMLNKKHOFFEF28CA,67.09,1.11,10.07,None,None,15
161,161,淡水區,房地(土地+建物)+車位,新北市淡水區中正路二段３６之３號九樓,6.21,商,None,None,1021014,土地2建物1車位1,...,13.69,850000,None,RPVOMLNKKHOFFEF78CA,41.73,3.93,4.02,None,None,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49591,5540,北投區,房地(土地+建物)+車位,臺北市北投區文林北路１３６號十八樓,27.36,住,None,None,1080322,土地1建物1車位2,...,30.02,0,None,RPRPMLNJOHOFFEA28CA,188.02,19.71,24.50,None,None,25
49624,5573,內湖區,房地(土地+建物),臺北市內湖區港墘路127巷12號13樓,23.56,住,None,None,1080326,土地1建物1車位0,...,0.0,0,None,RPWOMLNKNHOFFCA48CA,95.75,3.96,11.84,None,None,15
49633,5582,萬華區,房地(土地+建物)+車位,臺北市萬華區康定路６０號四樓之１,9.25,商,None,None,1080410,土地1建物1車位1,...,11.43,0,None,RPQOMLPKNHOFFBA97CA,72.47,1.90,9.08,None,None,15
49638,5587,文山區,房地(土地+建物)+車位,臺北市文山區育英街45巷8之1號11樓,30.69,住,None,None,1080330,土地1建物1車位2,...,79.88,0,None,RPVOMLOKNHOFFAA28CA,91.06,8.16,9.96,None,None,15


In [22]:
del df1_result['總樓層數(數字)']

In [23]:
df1_result

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號
2,2,板橋區,房地(土地+建物),新北市板橋區民生路三段246號16樓,7.83,住,None,None,1071214,土地2建物1車位0,...,None,0.0,0,含增建或未登記建物。,RPTNMLNKKHOFFAF08CA,109.02,0.00,13.33,None,None
8,8,板橋區,房地(土地+建物)+車位,新北市板橋區文化路一段３６０號二十三樓之１３,6.88,住,None,None,1071215,土地1建物2車位3,...,坡道平面,101.1,0,None,RPUOMLMLKHOFFAF28CA,45.86,0.00,10.06,None,None
151,151,新店區,房地(土地+建物)+車位,新北市新店區中央路127巷13號2樓之2,13.97,住,None,None,1071207,土地1建物1車位1,...,坡道平面,39.01,0,含增建或未登記建物。；,RPVNMLMKKHOFFCF38CA,87.37,1.92,11.74,None,None
160,160,淡水區,房地(土地+建物)+車位,新北市淡水區沙崙路一段１２２號七樓,21.9,住,None,None,1071214,土地1建物1車位2,...,坡道平面,56.22,0,建商與地主合建案。；,RPRNMLNKKHOFFEF28CA,67.09,1.11,10.07,None,None
161,161,淡水區,房地(土地+建物)+車位,新北市淡水區中正路二段３６之３號九樓,6.21,商,None,None,1021014,土地2建物1車位1,...,坡道機械,13.69,850000,None,RPVOMLNKKHOFFEF78CA,41.73,3.93,4.02,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49591,5540,北投區,房地(土地+建物)+車位,臺北市北投區文林北路１３６號十八樓,27.36,住,None,None,1080322,土地1建物1車位2,...,坡道平面,30.02,0,None,RPRPMLNJOHOFFEA28CA,188.02,19.71,24.50,None,None
49624,5573,內湖區,房地(土地+建物),臺北市內湖區港墘路127巷12號13樓,23.56,住,None,None,1080326,土地1建物1車位0,...,None,0.0,0,None,RPWOMLNKNHOFFCA48CA,95.75,3.96,11.84,None,None
49633,5582,萬華區,房地(土地+建物)+車位,臺北市萬華區康定路６０號四樓之１,9.25,商,None,None,1080410,土地1建物1車位1,...,升降平面,11.43,0,None,RPQOMLPKNHOFFBA97CA,72.47,1.90,9.08,None,None
49638,5587,文山區,房地(土地+建物)+車位,臺北市文山區育英街45巷8之1號11樓,30.69,住,None,None,1080330,土地1建物1車位2,...,坡道平面,79.88,0,None,RPVOMLOKNHOFFAA28CA,91.06,8.16,9.96,None,None


In [24]:
# 重設 index
df1_result = df1_result.reset_index(drop=True)

In [25]:
df1_result

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號
0,2,板橋區,房地(土地+建物),新北市板橋區民生路三段246號16樓,7.83,住,None,None,1071214,土地2建物1車位0,...,None,0.0,0,含增建或未登記建物。,RPTNMLNKKHOFFAF08CA,109.02,0.00,13.33,None,None
1,8,板橋區,房地(土地+建物)+車位,新北市板橋區文化路一段３６０號二十三樓之１３,6.88,住,None,None,1071215,土地1建物2車位3,...,坡道平面,101.1,0,None,RPUOMLMLKHOFFAF28CA,45.86,0.00,10.06,None,None
2,151,新店區,房地(土地+建物)+車位,新北市新店區中央路127巷13號2樓之2,13.97,住,None,None,1071207,土地1建物1車位1,...,坡道平面,39.01,0,含增建或未登記建物。；,RPVNMLMKKHOFFCF38CA,87.37,1.92,11.74,None,None
3,160,淡水區,房地(土地+建物)+車位,新北市淡水區沙崙路一段１２２號七樓,21.9,住,None,None,1071214,土地1建物1車位2,...,坡道平面,56.22,0,建商與地主合建案。；,RPRNMLNKKHOFFEF28CA,67.09,1.11,10.07,None,None
4,161,淡水區,房地(土地+建物)+車位,新北市淡水區中正路二段３６之３號九樓,6.21,商,None,None,1021014,土地2建物1車位1,...,坡道機械,13.69,850000,None,RPVOMLNKKHOFFEF78CA,41.73,3.93,4.02,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7367,5540,北投區,房地(土地+建物)+車位,臺北市北投區文林北路１３６號十八樓,27.36,住,None,None,1080322,土地1建物1車位2,...,坡道平面,30.02,0,None,RPRPMLNJOHOFFEA28CA,188.02,19.71,24.50,None,None
7368,5573,內湖區,房地(土地+建物),臺北市內湖區港墘路127巷12號13樓,23.56,住,None,None,1080326,土地1建物1車位0,...,None,0.0,0,None,RPWOMLNKNHOFFCA48CA,95.75,3.96,11.84,None,None
7369,5582,萬華區,房地(土地+建物)+車位,臺北市萬華區康定路６０號四樓之１,9.25,商,None,None,1080410,土地1建物1車位1,...,升降平面,11.43,0,None,RPQOMLPKNHOFFBA97CA,72.47,1.90,9.08,None,None
7370,5587,文山區,房地(土地+建物)+車位,臺北市文山區育英街45巷8之1號11樓,30.69,住,None,None,1080330,土地1建物1車位2,...,坡道平面,79.88,0,None,RPVOMLOKNHOFFAA28CA,91.06,8.16,9.96,None,None


In [26]:
# 交易年月日要改成西洋
df1_result['交易年月日'] = df1_result['交易年月日'].astype(str)

In [27]:
# 擷取年月日
# 年+1911
df1_result['年'] = df1_result['交易年月日'].str[0:3].astype(int)+1911
df1_result['月'] = df1_result['交易年月日'].str[3:5]
df1_result['日'] = df1_result['交易年月日'].str[5:]

In [28]:
df1_result['交易年月日_國曆'] = df1_result['年'].astype(str)+df1_result['月']+df1_result['日']

In [29]:
#　改成datetime型態
df1_result['交易年月日_國曆'] = pd.to_datetime(df1_result['交易年月日_國曆'], format='%Y-%m-%d')

In [30]:
df1_result['交易年月日'] = df1_result['交易年月日_國曆']

In [31]:
df1_result['交易年月日']

0      2018-12-14
1      2018-12-15
2      2018-12-07
3      2018-12-14
4      2013-10-14
          ...    
7367   2019-03-22
7368   2019-03-26
7369   2019-04-10
7370   2019-03-30
7371   2019-04-22
Name: 交易年月日, Length: 7372, dtype: datetime64[ns]

In [32]:
del df1_result['交易年月日_國曆']
del df1_result['年']
del df1_result['月']
del df1_result['日']

In [33]:
# 取除各地縣市
df1_result["city"] = df1_result['土地位置建物門牌'].str[0:3]

In [34]:
df1_result["city"].unique()

array(['新北市', '高雄市', '臺中市', '桃園市', '臺北市'], dtype=object)

In [35]:
df1_result

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,city
0,2,板橋區,房地(土地+建物),新北市板橋區民生路三段246號16樓,7.83,住,None,None,2018-12-14,土地2建物1車位0,...,0.0,0,含增建或未登記建物。,RPTNMLNKKHOFFAF08CA,109.02,0.00,13.33,None,None,新北市
1,8,板橋區,房地(土地+建物)+車位,新北市板橋區文化路一段３６０號二十三樓之１３,6.88,住,None,None,2018-12-15,土地1建物2車位3,...,101.1,0,None,RPUOMLMLKHOFFAF28CA,45.86,0.00,10.06,None,None,新北市
2,151,新店區,房地(土地+建物)+車位,新北市新店區中央路127巷13號2樓之2,13.97,住,None,None,2018-12-07,土地1建物1車位1,...,39.01,0,含增建或未登記建物。；,RPVNMLMKKHOFFCF38CA,87.37,1.92,11.74,None,None,新北市
3,160,淡水區,房地(土地+建物)+車位,新北市淡水區沙崙路一段１２２號七樓,21.9,住,None,None,2018-12-14,土地1建物1車位2,...,56.22,0,建商與地主合建案。；,RPRNMLNKKHOFFEF28CA,67.09,1.11,10.07,None,None,新北市
4,161,淡水區,房地(土地+建物)+車位,新北市淡水區中正路二段３６之３號九樓,6.21,商,None,None,2013-10-14,土地2建物1車位1,...,13.69,850000,None,RPVOMLNKKHOFFEF78CA,41.73,3.93,4.02,None,None,新北市
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7367,5540,北投區,房地(土地+建物)+車位,臺北市北投區文林北路１３６號十八樓,27.36,住,None,None,2019-03-22,土地1建物1車位2,...,30.02,0,None,RPRPMLNJOHOFFEA28CA,188.02,19.71,24.50,None,None,臺北市
7368,5573,內湖區,房地(土地+建物),臺北市內湖區港墘路127巷12號13樓,23.56,住,None,None,2019-03-26,土地1建物1車位0,...,0.0,0,None,RPWOMLNKNHOFFCA48CA,95.75,3.96,11.84,None,None,臺北市
7369,5582,萬華區,房地(土地+建物)+車位,臺北市萬華區康定路６０號四樓之１,9.25,商,None,None,2019-04-10,土地1建物1車位1,...,11.43,0,None,RPQOMLPKNHOFFBA97CA,72.47,1.90,9.08,None,None,臺北市
7370,5587,文山區,房地(土地+建物)+車位,臺北市文山區育英街45巷8之1號11樓,30.69,住,None,None,2019-03-30,土地1建物1車位2,...,79.88,0,None,RPVOMLOKNHOFFAA28CA,91.06,8.16,9.96,None,None,臺北市


In [36]:
# 按造時間排序 
# 由進到遠
final_result = df1_result.sort_values(by='交易年月日', ascending=False)

In [37]:
# Print dataframe
final_result

,_c0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,city
2306,12861,淡水區,房地(土地+建物)+車位,新北市淡水區中正東路二段１６５號二十七樓,93.0,住,None,None,2019-05-01,土地1建物1車位2,...,49.52,4600000,1.登記權利人為二人2.二車位未拆分單價,RPONNLRKNHOFFEF78CA,379.72,30.68,63.88,None,None,新北市
7339,5347,中山區,房地(土地+建物)+車位,臺北市中山區林森北路５８５號,9.49,商,None,None,2019-04-30,土地1建物2車位1,...,39.17,2800000,None,RPVSMLMLNHOFFCA78CA,51.19,2.44,4.59,None,None,臺北市
3895,11082,三民區,房地(土地+建物)+車位,高雄市三民區民族一路543巷8號七樓之1,18.19,住,None,None,2019-04-30,土地1建物1車位1,...,21.89,0,None,RPRNMLMKNHOFFDE28CA,77.46,11.22,0.00,None,None,高雄市
3893,11001,左營區,房地(土地+建物)+車位,高雄市左營區新莊一路２５號十二樓,10.39,住,None,None,2019-04-30,土地1建物1車位1,...,0.0,0,精緻裝潢,RPQPMLOJOHOFFEE68CA,77.46,22.03,0.00,None,None,高雄市
3862,10823,鼓山區,房地(土地+建物),高雄市鼓山區龍德路３４５號六樓,9.96,商,None,None,2019-04-30,土地1建物1車位0,...,0.0,0,None,RPXNMLNKNHOFFAE68CA,73.01,5.04,0.00,None,None,高雄市
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2128,淡水區,房地(土地+建物),新北市淡水區中正路二段３６之１號二十樓,6.29,商,None,None,2013-08-04,土地2建物1車位0,...,0.0,0,None,RPXNMLNLKHOFFEF98CA,44.50,2.68,4.83,None,None,新北市
916,5310,淡水區,房地(土地+建物),新北市淡水區中正路二段４０之１號十五樓,5.92,商,None,None,2013-08-01,土地2建物1車位0,...,0.0,0,None,RPTNMLMLLHOFFEF58CA,41.13,3.93,3.80,None,None,新北市
5550,3160,八德區,房地(土地+建物)+車位,桃園市八德區中正一路１６號六樓,19.61,商,None,None,2013-04-27,土地1建物1車位1,...,37.11,1150000,None,RPOOMLSLLHOFFFH09CA,83.27,0.00,6.36,None,None,桃園市
7012,2758,士林區,房地(土地+建物)+車位,臺北市士林區中山北路六段３５巷１８號五樓,43.92,住,None,None,2013-02-15,土地1建物1車位2,...,65.18,6400000,另有公設車位住戶共用,RPQNMLLJMHOFFEA57CA,171.06,15.06,12.36,None,None,臺北市


In [38]:
final_result.columns

Index(['_c0', '鄉鎮市區', '交易標的', '土地位置建物門牌', '土地移轉總面積平方公尺', '都市土地使用分區',
       '非都市土地使用分區', '非都市土地使用編定', '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態',
       '主要用途', '主要建材', '建築完成年月', '建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳',
       '建物現況格局-衛', '建物現況格局-隔間', '有無管理組織', '總價元', '單價元平方公尺', '車位類別',
       '車位移轉總面積(平方公尺)', '車位總價元', '備註', '編號', '主建物面積', '附屬建物面積', '陽台面積', '電梯',
       '移轉編號', 'city'],
      dtype='object')

In [39]:
final_result = final_result[["鄉鎮市區", "交易年月日", '建物型態', 'city']]

In [40]:
# 改欄位名稱
final_result.rename(columns={"鄉鎮市區": "district", "交易年月日": "date", "建物型態": "building_state"}, inplace=True)

/tmp/ipykernel_17150/4205334859.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result.rename(columns={"鄉鎮市區": "district", "交易年月日": "date", "建物型態": "building_state"}, inplace=True)


In [41]:
# Convert to pyspark 的 dataframe
sparkDF=spark.createDataFrame(final_result) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- district: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- building_state: string (nullable = true)
 |-- city: string (nullable = true)

+--------+-------------------+--------------------------+------+
|district|               date|            building_state|  city|
+--------+-------------------+--------------------------+------+
|  淡水區|2019-05-01 00:00:00|住宅大樓(11層含以上有電梯)|新北市|
|  中山區|2019-04-30 00:00:00|住宅大樓(11層含以上有電梯)|臺北市|
|  三民區|2019-04-30 00:00:00|住宅大樓(11層含以上有電梯)|高雄市|
|  左營區|2019-04-30 00:00:00|住宅大樓(11層含以上有電梯)|高雄市|
|  鼓山區|2019-04-30 00:00:00|住宅大樓(11層含以上有電梯)|高雄市|
|  汐止區|2019-04-29 00:00:00|住宅大樓(11層含以上有電梯)|新北市|
|  鼓山區|2019-04-29 00:00:00|住宅大樓(11層含以上有電梯)|高雄市|
|  中壢區|2019-04-29 00:00:00|住宅大樓(11層含以上有電梯)|桃園市|
|  岡山區|2019-04-29 00:00:00|住宅大樓(11層含以上有電梯)|高雄市|
|  三民區|2019-04-27 00:00:00|住宅大樓(11層含以上有電梯)|高雄市|
|  土城區|2019-04-26 00:00:00|住宅大樓(11層含以上有電梯)|新北市|
|  桃園區|2019-04-26 00:00:00|住宅大樓(11層含以上有電梯)|桃園市|
|  鼓山區|2019-04-25 00:00:00|住宅大樓(11層含以上有電梯)|高雄市|
|  汐止區|2019-04

In [42]:
# 將兩個欄位合併成json格式
df_sparkDataframe = sparkDF.select("city","date",to_json(struct("building_state","district")).alias("events"))

In [43]:
# Print dataframe
df_sparkDataframe.show(truncate=100)

+------+-------------------+-------------------------------------------------------------------+
|  city|               date|                                                             events|
+------+-------------------+-------------------------------------------------------------------+
|新北市|2019-05-01 00:00:00|{"building_state":"住宅大樓(11層含以上有電梯)","district":"淡水區"}|
|臺北市|2019-04-30 00:00:00|{"building_state":"住宅大樓(11層含以上有電梯)","district":"中山區"}|
|高雄市|2019-04-30 00:00:00|{"building_state":"住宅大樓(11層含以上有電梯)","district":"三民區"}|
|高雄市|2019-04-30 00:00:00|{"building_state":"住宅大樓(11層含以上有電梯)","district":"左營區"}|
|高雄市|2019-04-30 00:00:00|{"building_state":"住宅大樓(11層含以上有電梯)","district":"鼓山區"}|
|新北市|2019-04-29 00:00:00|{"building_state":"住宅大樓(11層含以上有電梯)","district":"汐止區"}|
|高雄市|2019-04-29 00:00:00|{"building_state":"住宅大樓(11層含以上有電梯)","district":"鼓山區"}|
|桃園市|2019-04-29 00:00:00|{"building_state":"住宅大樓(11層含以上有電梯)","district":"中壢區"}|
|高雄市|2019-04-29 00:00:00|{"building_state":"住宅大樓(11層含以上有電梯)","distric

In [44]:
# 再合併成 time_slots
df_sparkDataframe = df_sparkDataframe.select("city",to_json(struct("date","events")).alias("time_slots"))

In [45]:
# Print dataframe
df_sparkDataframe.show(truncate=100)

+------+---------------------------------------------------------------------------------------------------------------+
|  city|                                                                                                     time_slots|
+------+---------------------------------------------------------------------------------------------------------------+
|新北市|{"date":"2019-05-01T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\"...|
|臺北市|{"date":"2019-04-30T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\"...|
|高雄市|{"date":"2019-04-30T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\"...|
|高雄市|{"date":"2019-04-30T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\"...|
|高雄市|{"date":"2019-04-30T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\"...|
|新北市|{"date":"2019-04-29T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\"

In [46]:
# 依照各縣市合併
df_result_sparkDataframe = df_sparkDataframe.groupBy('city').agg(F.collect_list('time_slots').alias('time_slots'))

In [47]:
# Print dataframe
df_result_sparkDataframe.show(truncate=100)

+------+---------------------------------------------------------------------------------------------------------------+
|  city|                                                                                                     time_slots|
+------+---------------------------------------------------------------------------------------------------------------+
|新北市|[{"date":"2019-05-01T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\...|
|高雄市|[{"date":"2019-04-30T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\...|
|臺中市|[{"date":"2019-04-24T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\...|
|桃園市|[{"date":"2019-04-29T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\...|
|臺北市|[{"date":"2019-04-30T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\...|
+------+----------------------------------------------------------------------------------------------

In [48]:
# 分成兩個dataframe
df_slice_1, df_slice_2 = df_result_sparkDataframe.randomSplit([0.40, 0.60],seed=1234)

In [49]:
# Print dataframe
df_slice_1.show(truncate=100)

+------+---------------------------------------------------------------------------------------------------------------+
|  city|                                                                                                     time_slots|
+------+---------------------------------------------------------------------------------------------------------------+
|臺中市|[{"date":"2019-04-24T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\...|
|臺北市|[{"date":"2019-04-30T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\...|
+------+---------------------------------------------------------------------------------------------------------------+



In [50]:
# Print dataframe
df_slice_2.show(truncate=100)

+------+---------------------------------------------------------------------------------------------------------------+
|  city|                                                                                                     time_slots|
+------+---------------------------------------------------------------------------------------------------------------+
|新北市|[{"date":"2019-05-01T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\...|
|桃園市|[{"date":"2019-04-29T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\...|
|高雄市|[{"date":"2019-04-30T00:00:00.000Z","events":"{\"building_state\":\"住宅大樓(11層含以上有電梯)\",\"district\...|
+------+---------------------------------------------------------------------------------------------------------------+



In [51]:
# Output dataframe
df_slice_1.write.json('./result-part1.json')

In [52]:
# Output dataframe
df_slice_2.write.json('./result-part2.json')